# Load libraries

Важно: Не успях директно да копирам kaggle.json файла и затова първо се свързвам към Google Drive. Ще трябва да промените работната си директория по-долу, ако ще ползвате този код.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mainDirectory = "/content/drive" + "/MyDrive/Colab Notebooks/BirdCLEF" # Трябва да се смени това, ако подкарвате тетрадката във вашия драйв
fileName = mainDirectory + "/kaggle.json"

In [ ]:
! mkdir ~/.kaggle
! cp "{fileName}" ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Tensorflow utilities
!pip install -q tensorflow-addons==0.19.0
!pip install -q tensorflow-probability==0.19.0
!pip install -q tensorflow-io==0.32.0

# Other utilies
!pip install -q opencv-python-headless
!pip install -q librosa
!pip install -q scikit-learn

# Kaggle
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 29.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import tensorflow as tf 
import tensorflow_io as tfio
import zipfile
import csv
import pandas as pd
import numpy as np
import random
from glob import glob
from tqdm import tqdm
tqdm.pandas()  # enable progress bars in pandas operations
import gc

import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

# Import for visualization
import matplotlib as mpl
cmap = mpl.cm.get_cmap('coolwarm')
import matplotlib.pyplot as plt
import librosa.display as lid
import IPython.display as ipd
import cv2

# Import tensorflow
import tensorflow as tf
# Set logging level to avoid unnecessary messages
tf.get_logger().setLevel('ERROR')
# Set autograph verbosity to avoid unnecessary messages
tf.autograph.set_verbosity(0)

# Import required tensorflow modules
import tensorflow_io as tfio
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import tensorflow.keras.backend as K

<ipython-input-5-b54276914070>:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.12.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warni

# Download and extract data

In [ ]:
! kaggle competitions download birdclef-2023

100% 4.90G/4.91G [01:04<00:00, 63.8MB/s]
100% 4.91G/4.91G [01:04<00:00, 81.7MB/s]


In [ ]:
# Unzip the downloaded file
zip_ref = zipfile.ZipFile("birdclef-2023.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [ ]:
!ls train_audio

abethr1  blaplo1  carcha1  fatrav1  hartur1  mouwag1  rehblu1  spmthr1	wheslf1
abhori1  blbpuf2  carwoo1  fatwid1  helgui   ndcsun2  rehwea1  spwlap1	whhsaw1
abythr1  blcapa2  categr   fislov1  hipbab1  nobfly1  reisee2  squher1	whihel1
afbfly1  blfbus1  ccbeat1  fotdro5  hoopoe   norbro1  rerswa1  strher	whrshr1
afdfly1  blhgon1  chespa1  gabgos2  huncis1  norcro1  rewsta1  strsee1	witswa1
afecuc1  blhher1  chewea1  gargan   hunsun2  norfis1  rindov   stusta1	wlwwar
affeag1  blksaw1  chibat1  gbesta1  joygre1  norpuf1  rocmar2  subbus1	wookin1
afgfly1  blnmou1  chtapa3  gnbcam2  kerspa2  nubwoo1  rostur1  supsta1	woosan
afghor1  blnwea1  chucis1  gnhsun1  klacuc1  pabspa1  ruegls1  tacsun1	wtbeat1
afmdov1  bltapa1  cibwar1  gobbun1  kvbsun1  palfly2  rufcha2  tafpri1	yebapa1
afpfly1  bltbar1  cohmar1  gobsta5  laudov1  palpri1  sacibi2  tamdov1	yebbar1
afpkin1  bltori1  colsun2  gobwea1  lawgol   piecro1  sccsun2  thrnig1	yebduc1
afpwag1  blwlap1  combul2  golher1  lesmaw1  piekin1  s

# Configuration

In [ ]:
### Повечето неща са излишни. Копирах го директно от онази тетрадка в кагъл и ще махам ненужните неща с времето.

class CFG:
    # Verbosity level
    verbose = 0
    
    # Plot training history
    training_plot = True
    
    # Random seed
    seed = 42

    # Input image size and batch size
    img_size = [128, 384]
    batch_size = 32
    validation_thr = 2 # min number of samples in order to participate in train
    upsample_thr = 50 # min sample of each class in training set (upsample)
    
    # Audio duration, sample rate, and length
    duration = 10 # second
    sample_rate = 32000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    normalize = True

    # Training and validation size
    train_proportion = 0.8
    
    # Inference batch size, test time augmentation, and drop remainder
    infer_bs = 2
    tta = 1
    drop_remainder = True
    
    # Number of epochs
    epochs = 25

    # Pretraining, neck features, and final activation function
    pretrain = 'imagenet'
    neck_features = 0
    final_act = 'softmax'
    
    # Learning rate, optimizer, and scheduler
    lr = 1e-3
    scheduler = 'cos'
    optimizer = 'Adam' # AdamW, Adam
    
    # Loss function and label smoothing
    loss = 'CCE' # BCE, CCE
    label_smoothing = 0.05 # label smoothing
    
    # Data augmentation parameters
    augment=True
    
    # Time Freq masking
    freq_mask_prob=0.50
    num_freq_masks=1
    freq_mask_param=10
    time_mask_prob=0.50
    num_time_masks=2
    time_mask_param=25

    # Audio Augmentation Settings
    audio_augment_prob = 0.5
    
    mixup_prob = 0.65
    mixup_alpha = 0.5
    
    cutmix_prob = 0.65
    cutmix_alpha = 2.5
    
    timeshift_prob = 0.0
    
    gn_prob = 0.35

In [ ]:
tf.keras.utils.set_random_seed(CFG.seed)

# Explore the data

In [ ]:
dirpaths = []

In [ ]:
for dirpath, dirnames, filenames in os.walk("train_audio"):
    c = dirpath.split('/')
    if len(c) > 1:
        c = c[1]
    dirpaths.append((dirpath, len(filenames), c))
    print(f"There are {len(dirnames)} directories and {len(filenames)} audio files in '{dirpath}'.")

There are 264 directories and 0 audio files in 'train_audio'.
There are 0 directories and 51 audio files in 'train_audio/gycwar3'.
There are 0 directories and 172 audio files in 'train_audio/ratcis1'.
There are 0 directories and 136 audio files in 'train_audio/gargan'.
There are 0 directories and 7 audio files in 'train_audio/joygre1'.
There are 0 directories and 4 audio files in 'train_audio/palpri1'.
There are 0 directories and 10 audio files in 'train_audio/purgre2'.
There are 0 directories and 129 audio files in 'train_audio/hadibi1'.
There are 0 directories and 34 audio files in 'train_audio/kerspa2'.
There are 0 directories and 22 audio files in 'train_audio/yebgre1'.
There are 0 directories and 21 audio files in 'train_audio/norcro1'.
There are 0 directories and 29 audio files in 'train_audio/brican1'.
There are 0 directories and 28 audio files in 'train_audio/bkfruw1'.
There are 0 directories and 70 audio files in 'train_audio/reedov1'.
There are 0 directories and 3 audio files

In [ ]:
'train_audio/quailf1'.split('/')

['train_audio', 'quailf1']

In [ ]:
dirpaths.sort(key=lambda x: x[1], reverse = True)


In [ ]:
dirpaths[:10]

[('train_audio/thrnig1', 500, 'thrnig1'),
 ('train_audio/comsan', 500, 'comsan'),
 ('train_audio/wlwwar', 500, 'wlwwar'),
 ('train_audio/eaywag1', 500, 'eaywag1'),
 ('train_audio/barswa', 500, 'barswa'),
 ('train_audio/woosan', 486, 'woosan'),
 ('train_audio/combuz1', 477, 'combuz1'),
 ('train_audio/eubeat1', 437, 'eubeat1'),
 ('train_audio/hoopoe', 436, 'hoopoe'),
 ('train_audio/cohmar1', 425, 'cohmar1')]

In [ ]:
sum(map(lambda x: x[1], dirpaths))

16941

In [ ]:
sum(map(lambda x: x[1], dirpaths[:10]))

4761

In [ ]:
metadata = pd.read_csv("train_metadata.csv")
metadata

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,abethr1,[],['song'],4.3906,38.2788,Turdus tephronotus,African Bare-eyed Thrush,Rolf A. de By,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/128013,abethr1/XC128013.ogg
1,abethr1,[],['call'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363501,abethr1/XC363501.ogg
2,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363502,abethr1/XC363502.ogg
3,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/363503,abethr1/XC363503.ogg
4,abethr1,[],"['call', 'song']",-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/363504,abethr1/XC363504.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...
16936,yewgre1,[],[''],-1.2502,29.7971,Eurillas latirostris,Yellow-whiskered Greenbul,András Schmidt,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://xeno-canto.org/703472,yewgre1/XC703472.ogg
16937,yewgre1,[],[''],-1.2489,29.7923,Eurillas latirostris,Yellow-whiskered Greenbul,András Schmidt,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/703485,yewgre1/XC703485.ogg
16938,yewgre1,[],[''],-1.2433,29.7844,Eurillas latirostris,Yellow-whiskered Greenbul,András Schmidt,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/704433,yewgre1/XC704433.ogg
16939,yewgre1,[],[''],0.0452,36.3699,Eurillas latirostris,Yellow-whiskered Greenbul,Lars Lachmann,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/752974,yewgre1/XC752974.ogg


In [ ]:
assert len(metadata[metadata['primary_label'].isna()] ) + len(metadata[metadata['common_name'].isna()]) == 0

In [ ]:
num_labels = len(metadata[['primary_label', 'common_name']].drop_duplicates())
num_labels

264

In [ ]:
class2bird = metadata[['primary_label', 'common_name']].drop_duplicates().set_index('primary_label').to_dict()['common_name']

In [ ]:
taxonomy = pd.read_csv("eBird_Taxonomy_v2021.csv")
taxonomy

,TAXON_ORDER,CATEGORY,SPECIES_CODE,PRIMARY_COM_NAME,SCI_NAME,ORDER1,FAMILY,SPECIES_GROUP,REPORT_AS
0,1,species,ostric2,Common Ostrich,Struthio camelus,Struthioniformes,Struthionidae (Ostriches),Ostriches,NaN
1,6,species,ostric3,Somali Ostrich,Struthio molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
2,7,slash,y00934,Common/Somali Ostrich,Struthio camelus/molybdophanes,Struthioniformes,Struthionidae (Ostriches),NaN,NaN
3,8,species,grerhe1,Greater Rhea,Rhea americana,Rheiformes,Rheidae (Rheas),Rheas,NaN
4,14,species,lesrhe2,Lesser Rhea,Rhea pennata,Rheiformes,Rheidae (Rheas),NaN,NaN
...,...,...,...,...,...,...,...,...,...
16748,34694,species,slcgro1,Slate-colored Grosbeak,Saltator grossus,Passeriformes,Thraupidae (Tanagers and Allies),NaN,NaN
16749,34697,species,bltgro2,Black-throated Grosbeak,Saltator fuliginosus,Passeriformes,Thraupidae (Tanagers and Allies),NaN,NaN
16750,34698,spuh,saltat1,saltator sp.,Saltator sp.,Passeriformes,Thraupidae (Tanagers and Allies),NaN,NaN
16751,34699,spuh,passer1,passerine sp.,Passeriformes sp.,Passeriformes,NaN,Others,NaN


In [ ]:
train_dirs = list(map(lambda x: x[0], dirpaths[:-1]))
train_dirs[:10]

['train_audio/thrnig1',
 'train_audio/comsan',
 'train_audio/wlwwar',
 'train_audio/eaywag1',
 'train_audio/barswa',
 'train_audio/woosan',
 'train_audio/combuz1',
 'train_audio/eubeat1',
 'train_audio/hoopoe',
 'train_audio/cohmar1']

In [ ]:
assert len(class2bird) == num_labels

In [ ]:
dir_labels = list(map(lambda x: x[2], dirpaths[:-1]))
dir_labels = list(map(lambda x: class2bird[x], dir_labels))
dir_labels[:10]

['Thrush Nightingale',
 'Common Sandpiper',
 'Willow Warbler',
 'Western Yellow Wagtail',
 'Barn Swallow',
 'Wood Sandpiper',
 'Common Buzzard',
 'European Bee-eater',
 'Eurasian Hoopoe',
 'Common House-Martin']

In [ ]:
train_files = []
train_labels = []
for i in range(len(dir_labels)):
    for dirpath, dirnames, filenames in os.walk(train_dirs[i]):
        train_files = train_files + list(map(lambda x: train_dirs[i] + '/' + x , filenames))
        train_labels = train_labels + len(filenames) * [dir_labels[i]]

In [ ]:
train_files[:10], train_labels[:10]

(['train_audio/thrnig1/XC139013.ogg',
  'train_audio/thrnig1/XC697401.ogg',
  'train_audio/thrnig1/XC490123.ogg',
  'train_audio/thrnig1/XC484185.ogg',
  'train_audio/thrnig1/XC320149.ogg',
  'train_audio/thrnig1/XC469565.ogg',
  'train_audio/thrnig1/XC727114.ogg',
  'train_audio/thrnig1/XC507251.ogg',
  'train_audio/thrnig1/XC247794.ogg',
  'train_audio/thrnig1/XC559492.ogg'],
 ['Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale',
  'Thrush Nightingale'])

## Creating label encoder for the bird labels

This is a necessary first step in order to create One-Hot Encoding Later. The inverse transform will be used to get the Label name from the corresponding number.

In [ ]:
birds_label_encoder = LabelEncoder()

birds_label_encoder.fit(dir_labels)
birds_label_encoder.transform(dir_labels)

array([221,  74, 245, 228,  21, 247,  72,  89,  88,  73, 133,  71, 171,
        25, 111, 103, 183, 201,  70, 169,  62,  29, 220,  60,  85, 110,
        93,  95, 259, 235, 117,   2, 175, 164, 214, 186,  68, 128,  31,
       263, 255, 223,  13, 113, 222, 219, 114, 210, 163,   6, 216, 224,
        14,  42,  56,  97,  66,  79, 236, 192,  24, 257, 233,  10, 106,
       134, 176, 195, 129,  55, 248,  86,  53,  23, 120, 180, 211,   8,
       127, 174, 107, 165, 105, 196,  41,  77,   7, 212, 240, 172,  28,
       155, 194,  17, 112, 168,  61,  18, 177, 204, 206, 144,  50, 131,
        47,  33, 207, 148, 217,  76, 146, 250, 125, 161, 252,  22, 227,
        94,  59, 190, 198, 213,  78, 145, 208,   5,  48, 199,  58, 118,
        12, 147,  45, 132,  67, 260,  32, 244,   0, 100, 184, 157, 231,
        65, 262,  44, 116,  43,  11,  90, 185, 191, 153,  20, 108, 197,
        34, 232, 251,  30, 158,  49, 200, 187, 152,  82, 203, 202, 173,
       151, 156, 102, 154,  37, 249, 225, 115, 241, 139, 135, 11

## Display audio and spectrogram

In [ ]:
def load_audio(filepath):
    audio, sr = librosa.load(filepath)
    return audio, sr

In [ ]:
def get_spectrogram(audio):
    spec = librosa.feature.melspectrogram(y=audio, 
                                   sr=CFG.sample_rate, 
                                   n_mels=CFG.img_size[0],
                                   n_fft=CFG.nfft,
                                   hop_length=CFG.hop_length,
                                   fmax=CFG.fmax,
                                   fmin=CFG.fmin,
                                   )
    spec = librosa.power_to_db(spec, ref=1.0)
    return spec

In [ ]:
def display_audio(filepath, label):
    # Caption
    caption = f'File: {filepath} | Bird label: {label}'
    # Read audio file
    audio, sr = load_audio(filepath)
    # Keep fixed length audio
    ### audio = audio[:CFG.audio_len]
    # Spectrogram from audio
    spec = get_spectrogram(audio)
    # Display audio
    print("# Audio:")
    display(ipd.Audio(audio, rate=CFG.sample_rate))
    # Display spectrogram
    print('# Visualization:')
    fig, ax = plt.subplots(2, 1, figsize=(12, 2*3), sharex=True, tight_layout=True)
    fig.suptitle(caption)
    # Waveplot
    lid.waveshow(audio,
                 sr=CFG.sample_rate,
                 ax=ax[0],
                color= cmap(0.1))
    # Specplot
    lid.specshow(spec, 
                 sr = CFG.sample_rate, 
                 hop_length = CFG.hop_length,
                 n_fft=CFG.nfft,
                 fmin=CFG.fmin,
                 fmax=CFG.fmax,
                 x_axis = 'time', 
                 y_axis = 'mel',
                 cmap = 'coolwarm',
                 ax=ax[1])
    ax[0].set_xlabel('');
    fig.show()

In [86]:
display_audio(train_files[0], train_labels[0])

Output hidden; open in https://colab.research.google.com to view.

# Create train and validation datasets

In [ ]:
all_data = {"File": train_files,
            "Label": train_labels}
all_data = pd.DataFrame(all_data)

In [ ]:
all_data

,File,Label
0,train_audio/thrnig1/XC139013.ogg,Thrush Nightingale
1,train_audio/thrnig1/XC697401.ogg,Thrush Nightingale
2,train_audio/thrnig1/XC490123.ogg,Thrush Nightingale
3,train_audio/thrnig1/XC484185.ogg,Thrush Nightingale
4,train_audio/thrnig1/XC320149.ogg,Thrush Nightingale
...,...,...
16936,train_audio/crefra2/XC667443.ogg,Crested Francolin
16937,train_audio/lotcor1/XC317237.ogg,Long-tailed Cormorant
16938,train_audio/afpkin1/XC704863.ogg,African Pygmy Kingfisher
16939,train_audio/brtcha1/XC128120.ogg,Brown-tailed Chat


## Encoding the Label

In [ ]:
all_data.Label = birds_label_encoder.transform(all_data.Label)

In [ ]:
all_data

,File,Label
0,train_audio/thrnig1/XC139013.ogg,221
1,train_audio/thrnig1/XC697401.ogg,221
2,train_audio/thrnig1/XC490123.ogg,221
3,train_audio/thrnig1/XC484185.ogg,221
4,train_audio/thrnig1/XC320149.ogg,221
...,...,...
16936,train_audio/crefra2/XC667443.ogg,75
16937,train_audio/lotcor1/XC317237.ogg,137
16938,train_audio/afpkin1/XC704863.ogg,15
16939,train_audio/brtcha1/XC128120.ogg,54


## Filtering labels with too few examples

There are some labels with only 1 file. We should make sure they are in the training data instead of the validation data.

In [ ]:
def filter_data(df, thr=2):
    # Count the number of samples for each class
    counts = df.Label.value_counts()

    # Condition that selects classes with less than `thr` samples
    cond = df.Label.isin(counts[counts<thr].index.tolist())

    # Add a new column to select samples for cross validation
    df['Validation'] = True

    # Set cv = False for those class where there is samples less than thr
    df.loc[cond, 'Validation'] = False

    # Return the filtered dataframe
    return df

The rows with `Validation=False` will participate only in the training data.

In [ ]:
all_data = filter_data(all_data, thr=CFG.validation_thr)
all_data

,File,Label,Validation
0,train_audio/thrnig1/XC139013.ogg,221,True
1,train_audio/thrnig1/XC697401.ogg,221,True
2,train_audio/thrnig1/XC490123.ogg,221,True
3,train_audio/thrnig1/XC484185.ogg,221,True
4,train_audio/thrnig1/XC320149.ogg,221,True
...,...,...,...
16936,train_audio/crefra2/XC667443.ogg,75,False
16937,train_audio/lotcor1/XC317237.ogg,137,False
16938,train_audio/afpkin1/XC704863.ogg,15,False
16939,train_audio/brtcha1/XC128120.ogg,54,False


In [ ]:
all_data[all_data.Validation == False]

,File,Label,Validation
16934,train_audio/whctur2/XC444635.ogg,238,False
16935,train_audio/whhsaw1/XC289267.ogg,242,False
16936,train_audio/crefra2/XC667443.ogg,75,False
16937,train_audio/lotcor1/XC317237.ogg,137,False
16938,train_audio/afpkin1/XC704863.ogg,15,False
16939,train_audio/brtcha1/XC128120.ogg,54,False
16940,train_audio/yebsto1/XC337857.ogg,254,False


## Splitting train and validation sets

In [ ]:
train_only_data = all_data[all_data.Validation == False].drop(columns=["Validation"])

In [ ]:
to_split_data = all_data[all_data.Validation == True].drop(columns=["Validation"])

In [ ]:
train_data, test_data = train_test_split(to_split_data, train_size = CFG.train_proportion, random_state = CFG.seed)

In [ ]:
train_data = pd.concat([train_data, train_only_data])
train_data = shuffle(train_data, random_state=CFG.seed)
train_data = train_data.reset_index(drop=True)

test_data = test_data.reset_index(drop=True)

In [ ]:
train_data

,File,Label
0,train_audio/afecuc1/XC591478.ogg,6
1,train_audio/cohmar1/XC659483.ogg,73
2,train_audio/rbsrob1/XC646913.ogg,171
3,train_audio/comsan/XC325200.ogg,74
4,train_audio/blakit1/XC462026.ogg,25
...,...,...
13549,train_audio/hartur1/XC205711.ogg,119
13550,train_audio/comsan/XC582947.ogg,74
13551,train_audio/carcha1/XC515319.ogg,60
13552,train_audio/affeag1/XC253031.ogg,7


In [ ]:
test_data

,File,Label
0,train_audio/hoopoe/XC647839.ogg,88
1,train_audio/yewgre1/XC233084.ogg,263
2,train_audio/eaywag1/XC337614.ogg,228
3,train_audio/gyhbus1/XC514753.ogg,106
4,train_audio/reccuc1/XC682293.ogg,175
...,...,...
3382,train_audio/rebhor1/XC115455.ogg,157
3383,train_audio/slbgre1/XC401824.ogg,197
3384,train_audio/barswa/XC485999.ogg,21
3385,train_audio/subbus1/XC485979.ogg,216


## Upsampling the training set

... To be done

## Creating the datasets

In [ ]:
def audio_decoder(num_labels=264):
    def get_audio(filepath):
        file_bytes = tf.io.read_file(filepath)
        audio = tfio.audio.decode_vorbis(file_bytes) # decode .ogg file
        audio = tf.cast(audio, tf.float32)
        if tf.shape(audio)[1]>1: # stereo -> mono
            audio = audio[...,0:1]
        audio = tf.squeeze(audio, axis=-1)
        return audio

    def get_target(target):
        target = tf.reshape(target, [1])
        target = tf.cast(tf.one_hot(target, num_labels), tf.float32) 
        target = tf.reshape(target, [num_labels])
        return target
    
    def decode_with_labels(filepath, label):
        label = get_target(label)
        audio = get_audio(filepath)
        return audio, label
    
    return decode_with_labels

In [96]:
audio_decoder()(train_files[0], birds_label_encoder.transform([train_labels[0]]))

(<tf.Tensor: shape=(2659892,), dtype=float32, numpy=
 array([-1.6698212e-06, -1.6608433e-06,  8.1716399e-07, ...,
         6.1277626e-04,  4.6630282e-04,  2.3786877e-05], dtype=float32)>,
 <tf.Tensor: shape=(264,), dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
   

In [97]:
def build_dataset(paths, labels, num_labels = 264, repeat = False, shuffle = False):
    # Set TensorFlow AUTOTUNE option
    AUTO = tf.data.experimental.AUTOTUNE
    # Build the dataset
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    # Map the file names to the actual audio and the labels to the One-Hot encoding
    dataset = dataset.map(audio_decoder(num_labels), num_parallel_calls=AUTO)
    # Shuffle if enabled
    if shuffle:
        dataset = dataset.shuffle(shuffle, seed=CFG.seed)
    # Repeat if the option is enabled
    dataset = dataset.repeat() if repeat else dataset

    return dataset

In [98]:
train_dataset = build_dataset(train_data.File, train_data.Label, num_labels = num_labels, repeat = True, shuffle = True)
test_dataset = build_dataset(test_data.File, test_data.Label, num_labels = num_labels, repeat = False, shuffle = False)

In [99]:
for x in train_dataset.take(1):
    print(x)

for x in test_dataset.take(1):
    print(x)

(<tf.Tensor: shape=(6874593,), dtype=float32, numpy=
array([-1.0473224e-06,  2.5190622e-05,  2.9599485e-05, ...,
        8.5326947e-06, -4.0949087e-07, -2.0491756e-05], dtype=float32)>, <tf.Tensor: shape=(264,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0

## Какво остава?

За момента файловете се зареждат цели в dataset обектите. Планирам в самия модел да се добавят слоеве за:
- Отрязване на произволни 10 секунди (или добавяне на падинг за по-късите от 10 секунди записи) от записите - само при трениране
- Добавяне на data augmentation (на по-късен етап)
- Преобразуване в спектрограма
- Добавяне на TimeMask аугментация към спектрограмата (на по-късен етап)

В тази kaggle тетрадка първите 2 са направени като стъпки за създаването на datase-овете. Ако искате, можете да копирате кода:
https://www.kaggle.com/code/awsaf49/birdclef23-pretraining-is-all-you-need-train#EDA-%F0%9F%8E%A8
Секциите са `Data augmentation` за аугментацията и `Data Loader` за отрязването. Съответно за аугментацията може да се наложи да добавяте допълнителни стъпки в `build_dataset(...)` функцията